In [2]:
!pip install neo4j

In [2]:
#Data Loading

import logging
from neo4j import GraphDatabase

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Neo4j local connection details
NEO4J_URI = "neo4j://127.0.0.1:7687"  # Updated to match your instance
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "umbctest123"  # Password for umbc-test database

# Connect to Neo4j
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    logger.info("Successfully connected to Neo4j database.")
except Exception as e:
    logger.error(f"Failed to connect to Neo4j: {e}")
    driver = None


INFO:__main__:Successfully connected to Neo4j database.


In [3]:
# Check what nodes exist in the database
def check_database_nodes(driver, database="umbc-test"):
    with driver.session(database=database) as session:
        # Get all node labels and counts
        result = session.run("""
            MATCH (n) 
            RETURN labels(n) as NodeType, count(n) as Count
            ORDER BY Count DESC
        """)
        
        print(f"Node Types and Counts in '{database}' database:")
        print("-" * 40)
        for record in result:
            node_type = ', '.join(record["NodeType"])
            count = record["Count"]
            print(f"{node_type}: {count}")
        
        # Get total node count
        total_result = session.run("MATCH (n) RETURN count(n) as total")
        total = total_result.single()["total"]
        print(f"\nTotal nodes: {total}")

# Run the check on neo4j database (where your nodes actually are)
if driver:
    check_database_nodes(driver, "neo4j")
else:
    print("No database connection available")

Node Types and Counts in 'neo4j' database:
----------------------------------------
Course: 50
RequirementGroup: 12
Student: 10
Degree: 4
Faculty: 2
Term: 2

Total nodes: 80


In [4]:
# Check relationships in your database
def check_relationships(driver):
    with driver.session(database="neo4j") as session:
        # Get relationship types and counts
        result = session.run("""
            MATCH ()-[r]->() 
            RETURN type(r) as RelationshipType, count(r) as Count
            ORDER BY Count DESC
        """)
        
        print("Relationship Types and Counts:")
        print("=" * 40)
        total_relationships = 0
        for record in result:
            rel_type = record["RelationshipType"]
            count = record["Count"]
            total_relationships += count
            print(f"🔗 {rel_type}: {count}")
        
        print(f"\n🎯 Total relationships: {total_relationships}")
        
        if total_relationships == 0:
            print("\n⚠️  No relationships found! You need to import relationship files 08-17.")
        else:
            print(f"\n✅ Found {total_relationships} relationships!")
            
            # Show sample relationships
            print(f"\n📋 Sample relationships:")
            sample_result = session.run("MATCH ()-[r]->() RETURN r LIMIT 5")
            for record in sample_result:
                rel = record["r"]
                rel_type = rel.type
                start_node = rel.start_node
                end_node = rel.end_node
                start_label = list(start_node.labels)[0] if start_node.labels else "Unknown"
                end_label = list(end_node.labels)[0] if end_node.labels else "Unknown"
                start_id = start_node.get('id', start_node.get('name', 'No ID'))
                end_id = end_node.get('id', end_node.get('name', 'No ID'))
                print(f"  • {start_label}({start_id}) -[{rel_type}]-> {end_label}({end_id})")

# Check relationships
if driver:
    check_relationships(driver)
else:
    print("❌ No database connection available")


Relationship Types and Counts:
🔗 SIMILAR_CONTENT: 472
🔗 SIMILAR_DIFFICULTY: 372
🔗 LEADS_TO: 114
🔗 FULFILLS: 79
🔗 TEACHES: 71
🔗 OFFERED_IN: 62
🔗 PREREQUISITE_FOR: 33
🔗 COMPLETED: 25
🔗 ENROLLED_IN: 24
🔗 SIMILAR_LEARNING_STYLE: 18
🔗 PART_OF: 12
🔗 PURSUING: 11

🎯 Total relationships: 1293

✅ Found 1293 relationships!

📋 Sample relationships:
  • Unknown(No ID) -[PURSUING]-> Unknown(No ID)
  • Unknown(No ID) -[COMPLETED]-> Unknown(No ID)
  • Unknown(No ID) -[COMPLETED]-> Unknown(No ID)
  • Unknown(No ID) -[COMPLETED]-> Unknown(No ID)
  • Unknown(No ID) -[COMPLETED]-> Unknown(No ID)


In [5]:
# Better relationship viewer with actual node details
def show_detailed_relationships(driver, limit=10):
    with driver.session(database="neo4j") as session:
        print("🔍 Detailed Relationship Examples:")
        print("=" * 50)
        
        # Get sample relationships with actual node details
        result = session.run(f"""
            MATCH (start)-[r]->(end)
            RETURN 
                labels(start)[0] as StartType,
                start.id as StartID,
                type(r) as RelationshipType,
                labels(end)[0] as EndType,
                end.id as EndID
            LIMIT {limit}
        """)
        
        for i, record in enumerate(result, 1):
            start_type = record["StartType"]
            start_id = record["StartID"]
            rel_type = record["RelationshipType"]
            end_type = record["EndType"]
            end_id = record["EndID"]
            
            print(f"{i:2d}. {start_type}({start_id}) -[{rel_type}]-> {end_type}({end_id})")

# Show detailed relationships
if driver:
    show_detailed_relationships(driver)
else:
    print("❌ No database connection available")


🔍 Detailed Relationship Examples:
 1. Student(ZO28124) -[PURSUING]-> Degree(BA-ComputerScience-2)
 2. Student(ZO28124) -[COMPLETED]-> Course(CSEE 200)
 3. Student(ZO28124) -[COMPLETED]-> Course(CSLL 100-6)
 4. Student(ZO28124) -[COMPLETED]-> Course(BUUU 100)
 5. Student(ZO28124) -[COMPLETED]-> Course(BOOO 100-5)
 6. Student(ZO28124) -[ENROLLED_IN]-> Course(CSEE 100)
 7. Student(ZO28124) -[ENROLLED_IN]-> Course(CSZZ 200)
 8. Student(ZO28124) -[SIMILAR_LEARNING_STYLE]-> Student(WC43955)
 9. Student(WS68513) -[PURSUING]-> Degree(BA-ComputerScience-2)
10. Student(WS68513) -[ENROLLED_IN]-> Course(CSSS 200)


In [6]:
# Useful query functions for your hackathon application
class Neo4jQueries:
    def __init__(self, driver):
        self.driver = driver
    
    def get_student_info(self, student_id):
        """Get complete information about a student"""
        with self.driver.session(database="neo4j") as session:
            result = session.run("""
                MATCH (s:Student {id: $student_id})
                OPTIONAL MATCH (s)-[:PURSUING]->(d:Degree)
                OPTIONAL MATCH (s)-[:COMPLETED]->(c:Course)
                OPTIONAL MATCH (s)-[:ENROLLED_IN]->(e:Course)
                RETURN s, d, collect(DISTINCT c) as completed_courses, 
                       collect(DISTINCT e) as enrolled_courses
            """, student_id=student_id)
            
            record = result.single()
            if record:
                student = record["s"]
                degree = record["d"]
                completed = record["completed_courses"]
                enrolled = record["enrolled_courses"]
                
                print(f"🎓 Student: {student.get('name', 'Unknown')} ({student_id})")
                print(f"📚 Degree: {degree.get('name', 'None') if degree else 'None'}")
                print(f"✅ Completed Courses: {len([c for c in completed if c])}")
                print(f"📖 Currently Enrolled: {len([e for e in enrolled if e])}")
                return {
                    'student': dict(student),
                    'degree': dict(degree) if degree else None,
                    'completed_courses': [dict(c) for c in completed if c],
                    'enrolled_courses': [dict(e) for e in enrolled if e]
                }
            return None
    
    def get_course_recommendations(self, student_id, limit=5):
        """Get course recommendations based on completed courses"""
        with self.driver.session(database="neo4j") as session:
            result = session.run("""
                MATCH (s:Student {id: $student_id})-[:COMPLETED]->(completed:Course)
                MATCH (completed)-[:LEADS_TO]->(recommended:Course)
                WHERE NOT (s)-[:COMPLETED]->(recommended)
                AND NOT (s)-[:ENROLLED_IN]->(recommended)
                RETURN DISTINCT recommended.id as course_id, 
                       recommended.name as course_name,
                       count(*) as strength
                ORDER BY strength DESC
                LIMIT $limit
            """, student_id=student_id, limit=limit)
            
            recommendations = []
            for record in result:
                recommendations.append({
                    'course_id': record["course_id"],
                    'course_name': record["course_name"],
                    'strength': record["strength"]
                })
            return recommendations
    
    def find_similar_students(self, student_id, limit=5):
        """Find students with similar learning styles"""
        with self.driver.session(database="neo4j") as session:
            result = session.run("""
                MATCH (s1:Student {id: $student_id})-[:SIMILAR_LEARNING_STYLE]-(s2:Student)
                RETURN s2.id as student_id, s2.name as name
                LIMIT $limit
            """, student_id=student_id, limit=limit)
            
            similar_students = []
            for record in result:
                similar_students.append({
                    'student_id': record["student_id"],
                    'name': record["name"]
                })
            return similar_students

# Create query helper instance
if driver:
    queries = Neo4jQueries(driver)
    print("✅ Query helper functions ready!")
    print("\nAvailable functions:")
    print("- queries.get_student_info(student_id)")
    print("- queries.get_course_recommendations(student_id)")
    print("- queries.find_similar_students(student_id)")
else:
    print("❌ No database connection available")


✅ Query helper functions ready!

Available functions:
- queries.get_student_info(student_id)
- queries.get_course_recommendations(student_id)
- queries.find_similar_students(student_id)


In [9]:
if driver:
    recommendations = queries.get_course_recommendations("ZO28124")
    print(recommendations)

[{'course_id': 'CSDD 300', 'course_name': 'Computer Architecture I', 'strength': 2}, {'course_id': 'CSWW 300', 'course_name': 'Computer Networks Analysis', 'strength': 1}, {'course_id': 'CSRR 200', 'course_name': 'DevOps', 'strength': 1}, {'course_id': 'CSUU 200', 'course_name': 'Edge Computing', 'strength': 1}, {'course_id': 'CSVV 300', 'course_name': 'Data Structures Analysis', 'strength': 1}]


In [ ]:
# Demo: Test the query functions with student ZO28124
if driver:
    print("🧪 Testing Query Functions")
    print("=" * 40)
    
    # Test student info
    print("\n1️⃣ Student Information:")
    student_info = queries.get_student_info("ZO28124")
    
    # Test course recommendations
    print("\n2️⃣ Course Recommendations:")
    recommendations = queries.get_course_recommendations("ZO28124")
    if recommendations:
        for i, rec in enumerate(recommendations, 1):
            print(f"   {i}. {rec['course_id']}: {rec['course_name']} (strength: {rec['strength']})")
    else:
        print("   No recommendations found")
    
    # Test similar students
    print("\n3️⃣ Similar Students:")
    similar = queries.find_similar_students("ZO28124")
    if similar:
        for student in similar:
            print(f"   • {student['name']} ({student['student_id']})")
    else:
        print("   No similar students found")
        
    print("\n🎯 Your database is ready for hackathon development!")
else:
    print("❌ No database connection available")


🧪 Testing Query Functions

1️⃣ Student Information:
🎓 Student: Christine Thomas (ZO28124)
📚 Degree: Bachelor of Arts in Computer Science
✅ Completed Courses: 4
📖 Currently Enrolled: 2

2️⃣ Course Recommendations:


KeyError: 'title'